In [48]:
import sys
import polars as pl
import numpy as np
import pandas as pd
import snowflake.connector
import json
from datetime import datetime, timedelta
import os
import re
from snowflake.connector.pandas_tools import write_pandas
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd

In [49]:
counter = 0
while True:
    print(f"Intento {counter + 1}")
    if counter < 4:
        try:
            with open('credentials.json') as f:
                data_pass = json.load(f)

            pass_ = input("INGRESAR PASSCODE:")

            snowflake_connection = snowflake.connector.connect(
                user=data_pass['snow']['USER'],
                password=data_pass['snow']['PASS'],
                account=data_pass['snow']['ACCOUNT'],
                passcode=pass_,
                database='SANDBOX_PLUS',
                schema='DWH'
            )

            cursor = snowflake_connection.cursor()

            print('Correct Password - connected to SNOWFLAKE')

            break

        except FileNotFoundError:
            print("Error: 'credentials.json' file not found.")
            break
        except json.JSONDecodeError:
            print("Error: 'credentials.json' file is not valid JSON.")
            break
        except Exception as e:
            counter += 1
            print(f'Error: {e}')
            print('Incorrect Password - provide again')

    else:
        print('3 Intentos fallidos')
        break

Intento 1
Correct Password - connected to SNOWFLAKE


In [50]:
# mangold-cuenta-servicio@projecto-promos.iam.gserviceaccount.com

scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('C:\\Users\\leonardo.mangold\\PycharmProjects\\promos_inteligencia_negocio\\ft_promos_automatico\\leo_usuario_servicio_credenciales.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [51]:
url = 'https://docs.google.com/spreadsheets/d/1sZet9eBzOmokS1wBMw3lT46xJQQIWdO-fMk50toKV7A/edit#gid=775311320'
inducidas_google_drive = gc.open_by_url(url)

worksheet_1 = inducidas_google_drive.worksheet('Inducidas 2025 en adelante')

data = worksheet_1.get_all_values()

inducidas = pd.DataFrame(data[1:], columns=data[0])
inducidas.columns = [column.split('\n')[0] for column in inducidas.columns]

# AÑO / MES / /Item / PRECIO DE VENTA
inducidas.head(2)

,AÑO,MES,ESTADISTICO,Item,DESCRIPCIÓN,UNIDADES TOTALES,UNIDADES CERCANIAS,LOCALES INVOLUCRADOS,Costo (para emitir compra),PRECIO DE VENTA,...,,Valorizado PROMO,Num OC (Compra 1),Unds,Num OC (Compra 2),Unds,Total,Valorizado compras,Venta,Peso de venta sobre estimado
0,2025,Enero,262366000,1000524344,Oblea limon,"33,000","4,600",TODOS,14.9,25,...,,"$491,700.00",,,,,,,,
1,2025,Enero,262365000,1000524343,Oblea chocolate,"58,000","12,000",TODOS,14.9,25,...,,"$864,200.00",,,,,,,,


In [52]:
inducidas = inducidas[inducidas['Item'] != '']
inducidas = inducidas[inducidas['Item'] != '-']
inducidas['Item'] = list(map(lambda x: str(x).replace(' ', ''), inducidas['Item']))
inducidas['Item'] = inducidas['Item'].map(lambda x: str(x).strip())

month_dict = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

inducidas['PROM_FECHA_INICIO'] = inducidas.apply(lambda row: pd.Timestamp(f'{row["AÑO"]}-{month_dict[row["MES"]]}-01'), axis=1)

inducidas['PROM_FECHA_FIN'] = inducidas['PROM_FECHA_INICIO'] + MonthEnd(0)

inducidas

fecha_eventos_dict = {
        2168: '2024-01-01',
        2217: '2024-02-01',
        2236: '2024-03-01',
        2252: '2024-04-01',
        2271: '2024-05-01',
        2292: '2024-06-01',
        2293: '2024-07-01',
        2294: '2024-08-01',
        2295: '2024-09-01',
        2296: '2024-10-01',
        2297: '2024-11-01',
        2298: '2024-12-01',
        2463: '2025-01-01',
        2464: '2025-02-01',
        2465: '2025-03-01',
        2466: '2025-04-01',
        2467: '2025-05-01',
        2468: '2025-06-01',
        2469: '2025-07-01',
        2470: '2025-08-01',
        2471: '2025-09-01',
        2472: '2025-10-01',
        2473: '2025-11-01',
        2474: '2025-12-01'

    }
fecha_eventos_dict_reversed = {v: k for k, v in fecha_eventos_dict.items()}
fecha_eventos_df = pd.DataFrame(list(fecha_eventos_dict_reversed.items()), columns=['PROM_FECHA_INICIO', 'EVENTO_ID'])

inducidas['PROM_FECHA_INICIO'] = pd.to_datetime(inducidas['PROM_FECHA_INICIO'])
fecha_eventos_df['PROM_FECHA_INICIO'] = pd.to_datetime(fecha_eventos_df['PROM_FECHA_INICIO'])

inducidas = inducidas.merge(fecha_eventos_df, on = 'PROM_FECHA_INICIO', how = 'inner')

inducidas['PROM_FECHA_INICIO'] = inducidas['PROM_FECHA_INICIO'].dt.strftime('%d/%m/%Y')
inducidas['PROM_FECHA_FIN'] = inducidas['PROM_FECHA_FIN'].dt.strftime('%d/%m/%Y')

inducidas = inducidas[['PROM_FECHA_INICIO', 'PROM_FECHA_FIN', 'EVENTO_ID', 'PRECIO DE VENTA', 'Item']]

inducidas.rename({
    'Item':'ORIN',
    'PRECIO DE VENTA':'PROM_PVP_OFERTA'
    }, axis=1, inplace=True)

inducidas['PROM_PVP_OFERTA'].fillna(0, inplace=True)

inducidas.dropna(inplace=True)

inducidas

C:\Users\leonardo.mangold\AppData\Local\Temp\ipykernel_5408\3522122837.py:63: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  inducidas['PROM_PVP_OFERTA'].fillna(0, inplace=True)


,PROM_FECHA_INICIO,PROM_FECHA_FIN,EVENTO_ID,PROM_PVP_OFERTA,ORIN
0,01/01/2025,31/01/2025,2463,25,1000524344
1,01/01/2025,31/01/2025,2463,25,1000524343
2,01/01/2025,31/01/2025,2463,25,1000524342
3,01/01/2025,31/01/2025,2463,25,1000524341
4,01/01/2025,31/01/2025,2463,25,1000524340


In [53]:
query = '''
WITH CTE_1 AS (
    SELECT
        LGL.GEOG_LOCL_COD
    FROM
        MSTRDB.DWH.LU_GEOG_LOCAL AS LGL
        LEFT JOIN MSTRDB.DWH.LU_GEOG_TIPO_LOCAL AS LGTL ON LGTL.GEOG_TLOC_ID = LGL.GEOG_TLOC_ID
    WHERE
        LGL.GEOG_UNNG_ID = 2
        AND LGL.GEOG_LOCL_COD NOT IN (196, 197, 198)
)

SELECT
    DISTINCT
    '{inicio_snow}' AS PROM_FECHA_INICIO,
    '{fin_snow}' AS PROM_FECHA_FIN,
    LAA.ARTC_ARTC_ID,
    LGL.GEOG_LOCL_ID,
    0 AS PRONOSTICO_VENTA,
    0 AS STOCK_INICIAL_PROMO,
    {evento_id_snow} AS EVENTO_ID,
    {precio_oferta_snow} AS PROM_PVP_OFERTA,
    0 AS PROM_LOCAL_ACTIVO,
    0 AS PROM_ESTIBA
FROM
    MSTRDB.DWH.FT_STOCK AS FS
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FS.GEOG_LOCL_ID
WHERE
    LGL.GEOG_LOCL_COD IN (SELECT * FROM CTE_1)
    AND FS.TIEM_DIA_ID = CURRENT_DATE() - 1
    AND LAA.ORIN = '{orines_snow}'
'''

carga_inducidas = pd.DataFrame()

for orin, precio_oferta, inicio, fin, evento_id in zip(
    inducidas['ORIN'],
    inducidas['PROM_PVP_OFERTA'],
    inducidas['PROM_FECHA_INICIO'],
    inducidas['PROM_FECHA_FIN'],
    inducidas['EVENTO_ID']
        ):

    cursor.execute(query.format(
        precio_oferta_snow=precio_oferta,
        orines_snow=orin,
        inicio_snow=inicio,
        fin_snow=fin,
        evento_id_snow=evento_id
    ))

    df_aux = cursor.fetch_pandas_all()

    carga_inducidas = pd.concat([carga_inducidas, df_aux])

carga_inducidas_excel = carga_inducidas.copy(deep = True)

In [54]:
carga_inducidas_excel.head(2)
carga_inducidas_excel.dtypes

PROM_FECHA_INICIO      object
PROM_FECHA_FIN         object
ARTC_ARTC_ID            int32
GEOG_LOCL_ID            int16
PRONOSTICO_VENTA         int8
STOCK_INICIAL_PROMO      int8
EVENTO_ID               int16
PROM_PVP_OFERTA          int8
PROM_LOCAL_ACTIVO        int8
PROM_ESTIBA              int8
dtype: object

In [55]:
carga_inducidas_excel['PROM_FECHA_INICIO'] = pd.to_datetime(carga_inducidas_excel['PROM_FECHA_INICIO']).dt.strftime('%Y-%m-%d')
carga_inducidas_excel['PROM_FECHA_FIN'] = pd.to_datetime(carga_inducidas_excel['PROM_FECHA_FIN']).dt.strftime('%Y-%m-%d')

# Function to format dates
def format_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    return f"{date_obj.month}/{date_obj.day}/{date_obj.year}"

# List of columns to apply the formatting
date_columns = ['PROM_FECHA_INICIO', 'PROM_FECHA_FIN']

# Apply the formatting function to each column
for col in date_columns:
    carga_inducidas_excel[col] = carga_inducidas_excel[col].apply(format_date)

query = '''
SELECT
    GEOG_LOCL_ID,
    GEOG_LOCL_COD
FROM
    MSTRDB.DWH.LU_GEOG_LOCAL
'''

cursor.execute(query)
local_snow = cursor.fetch_pandas_all()

carga_inducidas_excel = carga_inducidas_excel.merge(local_snow, on='GEOG_LOCL_ID', how='inner')

query = '''
SELECT
    ARTC_ARTC_ID,
    ARTC_ARTC_COD
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO
'''

cursor.execute(query)
articulo_snow = cursor.fetch_pandas_all()

carga_inducidas_excel = carga_inducidas_excel.merge(articulo_snow, on='ARTC_ARTC_ID', how='inner')

carga_inducidas_excel = carga_inducidas_excel[
    pd.to_datetime(carga_inducidas_excel['PROM_FECHA_INICIO']) > datetime.today().replace(day=1) - relativedelta(days=1)
]

carga_inducidas_excel[[
    'PROM_FECHA_INICIO',
    'PROM_FECHA_FIN',
    'ARTC_ARTC_COD',
    'EVENTO_ID',
    'PRONOSTICO_VENTA',
    'STOCK_INICIAL_PROMO',
    'GEOG_LOCL_COD',
    'PROM_PVP_OFERTA',
    'PROM_LOCAL_ACTIVO',
    'PROM_ESTIBA'
]].to_excel(f"Cargar en Excel BI\\Inducidas - {datetime.today().strftime('%Y-%m-%d')}.xlsx", index=False)

print('Termina descargar_inducidas')

C:\Users\leonardo.mangold\AppData\Local\Temp\ipykernel_5408\1536981928.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  carga_inducidas_excel['PROM_FECHA_FIN'] = pd.to_datetime(carga_inducidas_excel['PROM_FECHA_FIN']).dt.strftime('%Y-%m-%d')


Termina descargar_inducidas


In [57]:
carga_inducidas_excel

,PROM_FECHA_INICIO,PROM_FECHA_FIN,ARTC_ARTC_ID,GEOG_LOCL_ID,PRONOSTICO_VENTA,STOCK_INICIAL_PROMO,EVENTO_ID,PROM_PVP_OFERTA,PROM_LOCAL_ACTIVO,PROM_ESTIBA,GEOG_LOCL_COD,ARTC_ARTC_COD
0,1/1/2025,1/31/2025,9358508,9,0,0,2463,25,0,0,134,262366000
1,1/1/2025,1/31/2025,9358508,12,0,0,2463,25,0,0,143,262366000
2,1/1/2025,1/31/2025,9358508,16,0,0,2463,25,0,0,147,262366000
3,1/1/2025,1/31/2025,9358508,84,0,0,2463,25,0,0,317,262366000
4,1/1/2025,1/31/2025,9358508,85,0,0,2463,25,0,0,318,262366000
...,...,...,...,...,...,...,...,...,...,...,...,...
422,1/1/2025,1/31/2025,9358504,140,0,0,2463,25,0,0,163,262362000
423,1/1/2025,1/31/2025,9358504,77,0,0,2463,25,0,0,310,262362000
424,1/1/2025,1/31/2025,9358504,78,0,0,2463,25,0,0,311,262362000
425,1/1/2025,1/31/2025,9358504,222,0,0,2463,25,0,0,165,262362000
